In [ ]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install diffusers transformers accelerate opencv-python gradio

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install huggingface

In [ ]:
from huggingface_hub import login
login()

In [3]:
import torch
import gradio as gr
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import random
import numpy as np
import cv2
from PIL import Image

#  Load ControlNet (Canny or Depth, swap model here)
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_canny",
    torch_dtype=torch.float16
)

#  Load base model (photorealistic)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V6.0_B1_noVAE",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)


MAX_IMAGE_RESOLUTION = 1024
MAX_NUM_IMAGES = 4
MAX_SEED = 2**32 - 1
FIXED_PROMPT = "clear sky, sunny day, architecture, FullHD, residential, photo realistic, hyperrealistic, super detailed, intricate, shadows, high dynamic range, reflecting window, raytracing"

def randomize_seed_fn(seed, randomize):
    return random.randint(0, MAX_SEED) if randomize else seed


def process(
    image,
    a_prompt,
    n_prompt,
    num_samples,
    image_resolution,
    num_steps,
    guidance_scale,
    seed,
    canny_low_threshold,
    canny_high_threshold
):
    if seed == 0:
        seed = random.randint(0, MAX_SEED)
    generator = torch.manual_seed(seed)


    raw = np.array(image.convert("RGB").resize((image_resolution, image_resolution)))
    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    detected_map = cv2.Canny(gray, canny_low_threshold, canny_high_threshold)
    detected_map = cv2.cvtColor(detected_map, cv2.COLOR_GRAY2RGB)
    control_image = Image.fromarray(detected_map)


    output = pipe(
        prompt=f"{FIXED_PROMPT}, {a_prompt}",
        negative_prompt=n_prompt,
        image=control_image,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=generator,
        num_images_per_prompt=num_samples,
    )

    return [img for img in output.images]


def launch_ui():
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column():
                image = gr.Image(label="Input Sketch/Image", type="pil")

                generate_button = gr.Button("Generate")
                with gr.Accordion("Advanced options", open=False):
                    num_samples = gr.Slider(label="Number of images", minimum=1, maximum=MAX_NUM_IMAGES, value=1, step=1)
                    image_resolution = gr.Slider(label="Image resolution", minimum=MAX_IMAGE_RESOLUTION, maximum=MAX_IMAGE_RESOLUTION, value=1024, step=256)
                    canny_low_threshold = gr.Slider(label="Canny low threshold", minimum=1, maximum=255, value=100, step=1)
                    canny_high_threshold = gr.Slider(label="Canny high threshold", minimum=1, maximum=255, value=200, step=1)
                    num_steps = gr.Slider(label="Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=30.0, value=9.0, step=0.1)
                    seed = gr.Slider(label="Seed", minimum=0, maximum=MAX_SEED, step=1, value=0)
                    randomize_seed = gr.Checkbox(label="Randomize seed", value=True)
                    a_prompt = gr.Textbox(label="Additional prompt", value="best quality, photo realistic, DSLR, 8k")
                    n_prompt = gr.Textbox(label="Negative prompt", value="blurry, cartoon, duplicate windows, poorly drawn, distorted")

            with gr.Column():
                result = gr.Gallery(label="Generated Image(s)", show_label=False, columns=2, object_fit="scale-down")

        inputs = [image, a_prompt, n_prompt, num_samples, image_resolution, num_steps, guidance_scale, seed, canny_low_threshold, canny_high_threshold]

        generate_button.click(
            fn=randomize_seed_fn,
            inputs=[seed, randomize_seed],
            outputs=seed,
            queue=False
        ).then(
            fn=process,
            inputs=inputs,
            outputs=result
        )

    demo.queue().launch(share=True)


launch_ui()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/transformers/m

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c7c99d9132af125abf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
